# 🔍 调试新网页版API - 查看选择的CIF文件

## 目的
测试新版网页端 (`api_router_extension.py`) 的 `/api/search` 接口选择了哪些CIF文件。

## 测试流程
1. 模拟网页端的完整流程
2. 查看实际获取的CIF文件列表
3. 验证文件是否存在
4. 检查元素和化学式

## 环境要求
- ✅ UV虚拟环境已重建
- ✅ 索引文件已恢复正常
- 不需要启动Web服务器（直接测试函数）

## 步骤 1：导入库并设置路径

In [1]:
import sys
from pathlib import Path
import pandas as pd

# 设置路径
repo_root = Path.cwd().parent
scripts_path = repo_root / "scripts"
sys.path.insert(0, str(scripts_path))

# 导入关键函数
from dara_adapter import prepare_phases_for_dara
from pymatgen.core import Composition, Structure

print("✅ 库导入成功")
print(f"📁 仓库根目录: {repo_root}")
print(f"📁 Scripts路径: {scripts_path}")

✅ 库导入成功
📁 仓库根目录: d:\Haiwen\Code_Repositories\dara
📁 Scripts路径: d:\Haiwen\Code_Repositories\dara\scripts


## 步骤 2：配置测试参数

**修改这里来测试不同的化学系统：**

In [3]:
# ============================================================================
# 🔧 修改这里：配置你的测试参数
# ============================================================================

# 数据库选择
DATABASE = "ICSD"  # 可选: "ICSD", "COD", "MP"

# 必需元素（化学系统）
REQUIRED_ELEMENTS = ["Y", "Mo", "O"]  # 修改为你要测试的元素

# 排除元素（可选）
EXCLUDE_ELEMENTS = []  # 例如: ["Fe", "Cu"]

# 最大相数
MAX_PHASES = 500

# ============================================================================

print("=" * 80)
print("🧪 测试配置")
print("=" * 80)
print(f"\n数据库: {DATABASE}")
print(f"必需元素: {REQUIRED_ELEMENTS}")
print(f"排除元素: {EXCLUDE_ELEMENTS if EXCLUDE_ELEMENTS else '无'}")
print(f"最大相数: {MAX_PHASES}")
print("\n" + "=" * 80)

🧪 测试配置

数据库: ICSD
必需元素: ['Y', 'Mo', 'O']
排除元素: 无
最大相数: 500



## 步骤 3：模拟网页端API调用

这一步模拟 `api_router_extension.py` 中的 `/api/search` 接口逻辑。

In [4]:
print("\n" + "=" * 80)
print("🌐 模拟网页端API逻辑")
print("=" * 80)

# 1. 选择索引文件路径（模拟 api_router_extension.py 的逻辑）
index_files = {
    "ICSD": repo_root / "indexes" / "icsd_index_filled.parquet",
    "COD": repo_root / "indexes" / "cod_index_filled.parquet",
    "MP": repo_root / "indexes" / "mp_index.parquet",
}

index_path = index_files[DATABASE]

print(f"\n📂 步骤 1/4: 选择索引文件")
print(f"   数据库: {DATABASE}")
print(f"   索引路径: {index_path}")
print(f"   文件存在: {'✅ 是' if index_path.exists() else '❌ 否'}")

# 2. 调用 prepare_phases_for_dara（这是网页端实际调用的函数）
print(f"\n🔍 步骤 2/4: 调用 prepare_phases_for_dara()")
print(f"   必需元素: {REQUIRED_ELEMENTS}")
print(f"   排除元素: {EXCLUDE_ELEMENTS}")
print(f"   最大相数: {MAX_PHASES}")

database_cifs = prepare_phases_for_dara(
    index_path=index_path,
    required_elements=REQUIRED_ELEMENTS,
    exclude_elements=EXCLUDE_ELEMENTS,
    max_phases=MAX_PHASES,
)

print(f"\n✅ 步骤 3/4: 获取结果")
print(f"   找到CIF文件: {len(database_cifs)} 个")

# 3. 检查路径类型
if len(database_cifs) > 0:
    first_path = Path(database_cifs[0])
    print(f"\n📍 步骤 4/4: 路径格式检查")
    print(f"   第一个CIF: {database_cifs[0]}")
    print(f"   是绝对路径: {'✅ 是' if first_path.is_absolute() else '❌ 否 (相对路径)'}")
    print(f"   文件存在: {'✅ 是' if first_path.exists() else '❌ 否'}")
else:
    print(f"\n⚠️ 步骤 4/4: 无法检查路径格式（没有找到CIF文件）")

print("\n" + "=" * 80)


🌐 模拟网页端API逻辑

📂 步骤 1/4: 选择索引文件
   数据库: ICSD
   索引路径: d:\Haiwen\Code_Repositories\dara\indexes\icsd_index_filled.parquet
   文件存在: ✅ 是

🔍 步骤 2/4: 调用 prepare_phases_for_dara()
   必需元素: ['Y', 'Mo', 'O']
   排除元素: []
   最大相数: 500

✅ 步骤 3/4: 获取结果
   找到CIF文件: 194 个

📍 步骤 4/4: 路径格式检查
   第一个CIF: D:\Haiwen\Code_Repositories\dara\icsd_cifs\cif\0\00\41\4111.cif
   是绝对路径: ✅ 是
   文件存在: ✅ 是


✅ 步骤 3/4: 获取结果
   找到CIF文件: 194 个

📍 步骤 4/4: 路径格式检查
   第一个CIF: D:\Haiwen\Code_Repositories\dara\icsd_cifs\cif\0\00\41\4111.cif
   是绝对路径: ✅ 是
   文件存在: ✅ 是



## 步骤 4：查看CIF文件列表

显示所有找到的CIF文件，验证文件存在性。

In [ ]:
if len(database_cifs) == 0:
    print("\n" + "=" * 80)
    print("⚠️ 没有找到任何CIF文件")
    print("=" * 80)
    print("\n可能原因:")
    print("  1. 索引中没有同时包含所有必需元素的结构")
    print("  2. 元素名称拼写错误（注意大小写）")
    print("  3. 数据库中确实不存在该化学系统的相")
    print("\n建议:")
    print("  - 尝试减少元素数量（例如只测试2个元素）")
    print("  - 切换其他数据库（COD/ICSD/MP）")
    print("  - 检查元素符号是否正确（如'O'不是'o'）")
    print("=" * 80)
else:
    print("\n" + "=" * 80)
    print(f"📋 找到的CIF文件列表 (共 {len(database_cifs)} 个)")
    print("=" * 80)
    
    # 显示前50个文件
    display_count = min(50, len(database_cifs))
    
    print(f"\n显示前 {display_count} 个文件:\n")
    
    for i, cif_path in enumerate(database_cifs[:display_count], 1):
        path_obj = Path(cif_path)
        filename = path_obj.name
        exists = path_obj.exists()
        status = "✓" if exists else "✗"
        
        # 显示格式：序号. 文件名 [存在状态]
        print(f"  {i:3d}. {filename:40s} [{status}]")
    
    if len(database_cifs) > display_count:
        print(f"\n  ... 还有 {len(database_cifs) - display_count} 个文件未显示")
    
    # 统计信息
    existing_count = sum(1 for p in database_cifs if Path(p).exists())
    missing_count = len(database_cifs) - existing_count
    
    print("\n" + "-" * 80)
    print("📊 文件存在性统计:")
    print(f"   存在: {existing_count} 个 ✓")
    print(f"   缺失: {missing_count} 个 ✗")
    
    if missing_count > 0:
        print(f"\n   ⚠️ 警告: 有 {missing_count} 个文件不存在！")
        print("   缺失文件示例:")
        for p in database_cifs:
            if not Path(p).exists():
                print(f"     - {p}")
                break
    
    print("=" * 80)


📋 找到的CIF文件列表 (共 194 个)

显示前 50 个文件:

    1. 4111.cif                                 [✓]
    2. 15535.cif                                [✓]
    3. 15814.cif                                [✓]
    4. 15815.cif                                [✓]
    5. 15903.cif                                [✓]
    6. 16394.cif                                [✓]
    7. 16574.cif                                [✓]
    8. 18311.cif                                [✓]
    9. 21107.cif                                [✓]
   10. 23722.cif                                [✓]
   11. 23811.cif                                [✓]
   12. 24031.cif                                [✓]
   13. 24032.cif                                [✓]
   14. 24033.cif                                [✓]
   15. 24322.cif                                [✓]
   16. 24559.cif                                [✓]
   17. 24577.cif                                [✓]
   18. 26190.cif                                [✓]
   19. 27202.cif          

: 

## 步骤 5：读取CIF元数据

读取前10个CIF文件，验证化学式和元素是否正确。

In [ ]:
if len(database_cifs) == 0:
    print("⚠️ 没有CIF文件可以检查元数据")
else:
    print("\n" + "=" * 80)
    print(f"🔬 CIF文件元数据分析 (前10个)")
    print("=" * 80)
    
    metadata_list = []
    
    for i, cif_path in enumerate(database_cifs[:10], 1):
        path_obj = Path(cif_path)
        
        if not path_obj.exists():
            print(f"\n{i:2d}. {path_obj.name}")
            print(f"    ❌ 文件不存在: {cif_path}")
            continue
        
        try:
            # 读取CIF结构
            structure = Structure.from_file(cif_path)
            
            # 提取信息
            formula = structure.composition.reduced_formula
            elements = sorted([str(el) for el in structure.composition.elements])
            num_sites = len(structure)
            spacegroup = structure.get_space_group_info()[0]
            
            metadata_list.append({
                'filename': path_obj.name,
                'formula': formula,
                'elements': ', '.join(elements),
                'sites': num_sites,
                'spacegroup': spacegroup
            })
            
            # 打印详细信息
            print(f"\n{i:2d}. {path_obj.name}")
            print(f"    化学式: {formula}")
            print(f"    元素: {', '.join(elements)}")
            print(f"    原子数: {num_sites}")
            print(f"    空间群: {spacegroup}")
            
            # 验证元素
            elements_set = set(elements)
            required_set = set(REQUIRED_ELEMENTS)
            
            if required_set.issubset(elements_set):
                print(f"    ✅ 包含所有必需元素: {REQUIRED_ELEMENTS}")
            else:
                missing = required_set - elements_set
                print(f"    ⚠️ 缺少元素: {missing}")
            
        except Exception as e:
            print(f"\n{i:2d}. {path_obj.name}")
            print(f"    ❌ 读取错误: {e}")
    
    # 生成汇总表格
    if metadata_list:
        print("\n" + "=" * 80)
        print("📊 元数据汇总表")
        print("=" * 80)
        
        df = pd.DataFrame(metadata_list)
        print("\n" + df.to_string(index=False))
        print("\n" + "=" * 80)

## 步骤 6：总结与建议

基于测试结果的分析和建议。

In [ ]:
print("\n" + "=" * 80)
print("📋 测试总结报告")
print("=" * 80)

print(f"\n📌 测试配置:")
print(f"   数据库: {DATABASE}")
print(f"   必需元素: {REQUIRED_ELEMENTS}")
print(f"   排除元素: {EXCLUDE_ELEMENTS if EXCLUDE_ELEMENTS else '无'}")
print(f"   最大相数: {MAX_PHASES}")

print(f"\n📊 测试结果:")
print(f"   找到CIF文件: {len(database_cifs)} 个")

if len(database_cifs) > 0:
    existing = sum(1 for p in database_cifs if Path(p).exists())
    absolute = sum(1 for p in database_cifs if Path(p).is_absolute())
    
    print(f"   文件存在: {existing}/{len(database_cifs)} 个")
    print(f"   绝对路径: {absolute}/{len(database_cifs)} 个")

print(f"\n✅ 检查项:")

# 检查1: 是否找到CIF
if len(database_cifs) == 0:
    print(f"  ❌ 1. 未找到任何CIF文件")
    print(f"       → 索引中可能没有包含所有元素 {REQUIRED_ELEMENTS} 的结构")
else:
    print(f"  ✅ 1. 成功找到 {len(database_cifs)} 个CIF文件")

# 检查2: 文件存在性
if len(database_cifs) > 0:
    missing = [p for p in database_cifs if not Path(p).exists()]
    if missing:
        print(f"  ⚠️ 2. 有 {len(missing)} 个CIF文件不存在")
        print(f"       → 第一个缺失文件: {missing[0]}")
    else:
        print(f"  ✅ 2. 所有CIF文件都存在")

# 检查3: 路径格式
if len(database_cifs) > 0:
    relative = [p for p in database_cifs if not Path(p).is_absolute()]
    if relative:
        print(f"  ⚠️ 3. 有 {len(relative)} 个相对路径")
        print(f"       → 可能导致网页端找不到文件")
        print(f"       → 第一个相对路径: {relative[0]}")
    else:
        print(f"  ✅ 3. 所有路径都是绝对路径")

print(f"\n💡 下一步建议:")

if len(database_cifs) == 0:
    print(f"  ⚠️ 需要调整搜索条件:")
    print(f"     1. 尝试减少元素数量（如只用2个元素）")
    print(f"     2. 切换其他数据库:")
    print(f"        - ICSD: 无机晶体结构数据库")
    print(f"        - COD: 开放晶体学数据库")
    print(f"        - MP: Materials Project（理论+实验）")
    print(f"     3. 检查元素符号拼写（注意大小写）")
elif len(database_cifs) > 0:
    print(f"  ✅ 网页端API应该能正常获取这些CIF文件")
    print(f"  ")
    print(f"  如果网页端仍然显示'No phases identified'，可能原因:")
    print(f"     1. XRD图谱数据质量问题（噪音、峰强度）")
    print(f"     2. 波长设置不正确（默认Cu Kα）")
    print(f"     3. 仪器参数不匹配")
    print(f"     4. search_phases 搜索参数过于严格")
    print(f"  ")
    print(f"  建议:")
    print(f"     - 在streamlined_phase_analysis.ipynb中用相同元素测试")
    print(f"     - 检查XRD图谱文件是否正确")
    print(f"     - 查看服务器日志获取详细错误信息")

print("\n" + "=" * 80)
print("✅ 调试完成！")
print("=" * 80)